In [9]:
import panel as pn
pn.extension('plotly')

The ``HoloViews`` pane renders HoloViews plots with one of the plotting backends supported by HoloViews. It supports the regular HoloViews widgets for exploring the key dimensions of a ``HoloMap`` or ``DynamicMap``, but is more flexible than the native HoloViews widgets since it also allows customizing widget types and their position relative to the plot.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``backend``** (str): Any of the supported HoloViews backends ('bokeh', 'matplotlib', or 'plotly')
* **``center``** (boolean, default=False): Whether to center the plot
* **``linked_axes``** (boolean, default=True): Whether to link axes across plots in a panel layout
* **``object``** (object): The HoloViews object being displayed
* **``widget_location``** (str): Where to lay out the widget relative to the plot 
* **``widget_layout``** (ListPanel type): The object to lay the widgets out in, one of ``Row``, ``Column`` or ``WidgetBox``
* **``widget_type``** (str): Whether to generate individual widgets for each dimension, or to use a global linear scrubber with dimensions concatenated.
* **``widgets``** (dict): A mapping from dimension name to a widget class, instance, or dictionary of overrides to modify the default widgets.

##### Display

* **``default_layout``** (pn.layout.Panel, default=Row): Layout to wrap the plot and widgets in

___

The `panel` function will automatically convert any ``HoloViews`` object into a displayable panel, while keeping all of its interactive features:

In [12]:
import numpy as np
import holoviews as hv

box = hv.BoxWhisker((np.random.randint(0, 10, 100), np.random.randn(100)), 'Group').sort()

hv_layout = pn.panel(box)
hv_layout

HoloViews(BoxWhisker)

By setting the pane's ``object`` the plot can be updated like all other pane objects:

In [13]:
hv_layout.object = hv.Violin(box).opts(violin_color='Group', cmap='Category20')

ImportError: univariate_kde operation requires SciPy to be installed.

### Widgets

HoloViews natively renders plots with widgets if a HoloMap or DynamicMap declares any key dimensions. Unlike Panel's ``interact`` functionality, this approach efficiently updates just the data inside a plot instead of replacing it entirely. Calling ``pn.panel`` on the DynamicMap will return a ``Row`` layout (configurable via the ``default_layout`` option), which is equivalent to calling ``pn.pane.HoloViews(dmap).layout``:

In [15]:
import pandas as pd
import hvplot.pandas
import holoviews.plotting.bokeh

def sine(frequency=1.0, amplitude=1.0, function='sin'):
    xs = np.arange(200)/200*20.0
    ys = amplitude*getattr(np, function)(frequency*xs)
    return pd.DataFrame(dict(y=ys), index=xs).hvplot()

dmap = hv.DynamicMap(sine, kdims=['frequency', 'amplitude', 'function']).redim.range(
    frequency=(0.1, 10), amplitude=(1, 10)).redim.values(function=['sin', 'cos', 'tan'])

hv_panel = pn.panel(dmap)

hv_panel.pprint()

Row
    [0] HoloViews(DynamicMap)
    [1] Column
        [0] WidgetBox
            [0] FloatSlider(end=10, margin=(20, 20, 5, 20), name='frequency', start=0.1, value=0.1, width=250)
            [1] IntSlider(end=10, margin=(0, 20, 5, 20), name='amplitude', start=1, value=1, width=250)
            [2] Select(margin=(5, 20, 20, 20), name='function', options=['sin', 'cos', 'tan'], value='sin', width=250)
        [1] VSpacer()


We can see the widgets generated for each of the dimensions and arrange them any way we like, e.g. by unpacking them into a ``Row``:

In [16]:
widgets = hv_panel[1]

pn.Column(
    pn.Row(*widgets),
    hv_panel[0])

Column
    [0] Row
        [0] WidgetBox
            [0] FloatSlider(end=10, margin=(20, 20, 5, 20), name='frequency', start=0.1, value=0.1, width=250)
            [1] IntSlider(end=10, margin=(0, 20, 5, 20), name='amplitude', start=1, value=1, width=250)
            [2] Select(margin=(5, 20, 20, 20), name='function', options=['sin', 'cos', 'tan'], value='sin', width=250)
        [1] VSpacer()
    [1] HoloViews(DynamicMap)

However, more conveniently the HoloViews pane offers options to lay out the plot and widgets in a number of preconfigured arrangements using the ``center`` and ``widget_location`` parameters.

In [17]:
pn.panel(dmap, center=True, widget_location='right_bottom')

Row
    [0] HSpacer()
    [1] HoloViews(DynamicMap, center=True, widget_location='right_bottom')
    [2] HSpacer()
    [3] Column
        [0] VSpacer()
        [1] WidgetBox
            [0] FloatSlider(end=10, margin=(20, 20, 5, 20), name='frequency', start=0.1, value=0.1, width=250)
            [1] IntSlider(end=10, margin=(0, 20, 5, 20), name='amplitude', start=1, value=1, width=250)
            [2] Select(margin=(5, 20, 20, 20), name='function', options=['sin', 'cos', 'tan'], value='sin', width=250)

The ``widget_location`` parameter accepts all of the following options:
    
    ['left', 'bottom', 'right', 'top', 'top_left', 'top_right', 'bottom_left',
     'bottom_right', 'left_top', 'left_bottom', 'right_top', 'right_bottom']

#### Customizing widgets

As we saw above, the HoloViews pane will automatically try to generate appropriate widgets for the type of data, usually defaulting to ``DiscreteSlider`` and ``Select`` widgets. This behavior can be modified by providing a dictionary of ``widgets`` by dimension name.  The values of this dictionary can override the default widget in one of three ways:

* Supplying a ``Widget`` instance
* Supplying a compatible ``Widget`` type
* Supplying a dictionary of ``Widget`` parameter overrides
    
``Widget`` instances will be used as they are supplied and are expected to provide values matching compatible with the values defined on HoloMap/DynamicMap. Similarly if a ``Widget`` type is supplied it should be discrete if the parameter space defines a discrete set of values.  If the defined parameter space is continuous, on the other hand, it may supply any valid value.

In the example below the 'amplitude' dimension is overridden with an explicit ``Widget`` instance, the 'function' dimension is overridden with a RadioButtonGroup letting us toggle between the different functions, and lastly the 'value' parameter on the 'frequency' widget is overridden to change the initial value:

In [18]:
hv_panel = pn.pane.HoloViews(dmap, widgets={
    'amplitude': pn.widgets.LiteralInput(value=1., type=(float, int)),
    'function': pn.widgets.RadioButtonGroup,
    'frequency': {'value': 5}
}).layout

### Switching backends

The ``HoloViews`` pane will default to the Bokeh backend if no backend has been loaded, but you can override the backend as needed.

In [19]:
import holoviews.plotting.mpl
import holoviews.plotting.plotly

hv_pane = pn.pane.HoloViews(dmap, backend='matplotlib')
hv_pane

ModuleNotFoundError: No module named 'matplotlib'

The ``backend``, like all other parameters, can be modified after the fact.  To demonstrate, we can set up a select widget to toggle between backends for the above plot:

In [ ]:
backend_select = pn.widgets.RadioButtonGroup(name='Backend Selector:', options=['bokeh', 'matplotlib', 'plotly'])
backend_select.link(hv_pane[0], value='backend')
backend_select